# Patient Data Query

This notebook queries patient data from multiple databases for a given prontuario.

## Tables Queried:
- **Clinisys Silver Layer** (clinisys_all.duckdb):
  - view_micromanipulacao
  - view_micromanipulacao_oocitos
  - view_tratamentos
  - view_embrioes_congelados
- **Finops Silver Layer** (huntington_data_lake.duckdb):
  - silver.mesclada_vendas


In [1]:
import duckdb
import pandas as pd
import os
from datetime import datetime

# Set pandas display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)


## Input: Patient Prontuario


In [2]:
def get_clinisys_connection():
    """Get connection to clinisys_all database"""
    clinisys_db_path = os.path.join('..', '..', 'database', 'clinisys_all.duckdb')
    return duckdb.connect(clinisys_db_path, read_only=True)

def get_huntington_connection():
    """Get connection to huntington_data_lake database"""
    huntington_db_path = os.path.join('..', '..', 'database', 'huntington_data_lake.duckdb')
    return duckdb.connect(huntington_db_path, read_only=True)


## Database Connections


In [3]:
def query_micromanipulacao(conn, prontuario):
    """Query view_micromanipulacao for a specific prontuario"""
    print(f"\n=== VIEW_MICROMANIPULACAO ===")
    query = f"""
    SELECT * FROM silver.view_micromanipulacao 
    WHERE prontuario = {prontuario}
    ORDER BY codigo_ficha DESC
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in view_micromanipulacao")
    # if not df.empty:
    #     display(df)
    # else:
    #     print("No records found.")
    return df

def query_micromanipulacao_oocitos(conn, prontuario):
    """Query view_micromanipulacao_oocitos for a specific prontuario"""
    print(f"\n=== VIEW_MICROMANIPULACAO_OOCITOS ===")
    
    # First get codigo_ficha from view_micromanipulacao for this prontuario
    codigos_query = f"""
    SELECT codigo_ficha FROM silver.view_micromanipulacao 
    WHERE prontuario = {prontuario}
    """
    
    codigos_df = conn.execute(codigos_query).df()
    if not codigos_df.empty:
        codigo_list = codigos_df['codigo_ficha'].tolist()
        placeholders = ','.join(['?'] * len(codigo_list))
        
        oocitos_query = f"""
        SELECT * FROM silver.view_micromanipulacao_oocitos
        WHERE id_micromanipulacao IN ({placeholders})
        ORDER BY id_micromanipulacao, id
        """
        
        df = conn.execute(oocitos_query, codigo_list).df()
        print(f"Found {len(df)} records in view_micromanipulacao_oocitos")
        # if not df.empty:
        #     display(df)
        # else:
        #     print("No records found.")
    else:
        print("No codigo_ficha found for this prontuario in view_micromanipulacao.")
        df = pd.DataFrame()
    
    return df


In [4]:
def query_biopsy_pgta_timeline(conn, prontuario):
    """Query biopsy and PGT-A timeline data for a specific prontuario"""
    print(f"\n=== BIOPSY PGT-A TIMELINE ===")
    query = f"""
    SELECT * FROM gold.biopsy_pgta_timeline 
    WHERE prontuario = {prontuario}
    ORDER BY period_month DESC
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in biopsy_pgta_timeline")
    # if not df.empty:
    #     display(df)
    # else:
    #     print("No records found.")
    return df

def query_billing_timeline(conn, prontuario):
    """Query billing timeline data for a specific prontuario"""
    print(f"\n=== BILLING TIMELINE ===")
    query = f"""
    SELECT * FROM gold.billing_timeline 
    WHERE prontuario = {prontuario}
    ORDER BY period_month DESC
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in billing_timeline")
    # if not df.empty:
    #     display(df)
    # else:
    #     print("No records found.")
    return df


In [5]:
def query_tratamentos(conn, prontuario):
    """Query view_tratamentos for a specific prontuario"""
    print(f"\n=== VIEW_TRATAMENTOS ===")
    query = f"""
    SELECT * FROM silver.view_tratamentos 
    WHERE prontuario = {prontuario}
    ORDER BY id DESC
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in view_tratamentos")
    # if not df.empty:
    #     display(df)
    # else:
    #     print("No records found.")
    return df

def query_embrioes_congelados(conn, prontuario):
    """Query view_embrioes_congelados for a specific prontuario"""
    print(f"\n=== VIEW_EMBRIOES_CONGELADOS ===")
    query = f"""
    SELECT * FROM silver.view_embrioes_congelados 
    WHERE prontuario = {prontuario}
    ORDER BY id DESC
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in view_embrioes_congelados")
    # if not df.empty:
    #     display(df)
    # else:
    #     print("No records found.")
    return df

def query_congelamentos_embrioes(conn, prontuario):
    """Query view_congelamentos_embrioes for a specific prontuario"""
    print(f"\n=== VIEW_CONGELAMENTOS_EMBRIOES ===")
    query = f"""
    SELECT * FROM silver.view_congelamentos_embrioes 
    WHERE prontuario = {prontuario}
    ORDER BY id DESC
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in view_congelamentos_embrioes")
    # if not df.empty:
    #     display(df)
    # else:
    #     print("No records found.")
    return df

def query_congelamentos_ovulos(conn, prontuario):
    """Query view_congelamentos_ovulos for a specific prontuario"""
    print(f"\n=== VIEW_CONGELAMENTOS_OVULOS ===")
    query = f"""
    SELECT * FROM silver.view_congelamentos_ovulos 
    WHERE prontuario = {prontuario}
    ORDER BY id DESC
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in view_congelamentos_ovulos")
    # if not df.empty:
    #     display(df)
    # else:
    #     print("No records found.")
    return df

def query_congelamentos_semen(conn, prontuario):
    """Query view_congelamentos_semen for a specific prontuario"""
    print(f"\n=== VIEW_CONGELAMENTOS_SEMEN ===")
    query = f"""
    SELECT * FROM silver.view_congelamentos_semen 
    WHERE prontuario = {prontuario}
    ORDER BY id DESC
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in view_congelamentos_semen")
    # if not df.empty:
    #     display(df)
    # else:
    #     print("No records found.")
    return df

def query_mesclada_vendas(conn, prontuario):
    """Query silver.mesclada_vendas for a specific prontuario"""
    print(f"\n=== SILVER.MESCLADA_VENDAS ===")
    query = f"""
    SELECT * FROM silver.mesclada_vendas 
    WHERE prontuario = {prontuario}
    ORDER BY "DT Emissao" DESC
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in silver.mesclada_vendas")
    # if not df.empty:
    #     display(df)
    # else:
    #     print("No records found.")
    return df


In [6]:
def query_comprehensive_timeline(conn, prontuario):
    """Query comprehensive timeline data for a specific prontuario"""
    print(f"\n=== COMPREHENSIVE BIOPSY PGT-A TIMELINE TIMELINE ===")
    query = f"""
    SELECT * FROM gold.comprehensive_biopsy_pgta_timeline 
    WHERE prontuario = {prontuario}
    ORDER BY period_month DESC
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in comprehensive_biopsy_pgta_timeline")
    # if not df.empty:
    #     display(df)
    # else:
    #     print("No records found.")
    return df


In [7]:
def query_resumed_timeline(conn, prontuario):
    """Query resumed timeline data for a specific prontuario (most recent record only)"""
    print(f"\n=== RESUMED BIOPSY PGT-A TIMELINE TIMELINE (MOST RECENT RECORD) ===")
    query = f"""
    SELECT * FROM gold.resumed_biopsy_pgta_timeline 
    WHERE prontuario = {prontuario}
    ORDER BY period_month DESC
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in resumed_biopsy_pgta_timeline")
    # if not df.empty:
    #     display(df)
    # else:
    #     print("No records found.")
    return df


In [8]:
def query_embryoscope_timeline(conn, prontuario):
    """Query embryoscope timeline data for a specific prontuario"""
    print(f"\n=== EMBRYOSCOPE TIMELINE ===")
    query = f"""
    SELECT * FROM gold.embryoscope_timeline 
    WHERE prontuario = {prontuario}
    ORDER BY period_month DESC
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in resumed_biopsy_pgta_timeline")
    # if not df.empty:
    #     display(df)
    # else:
    #     print("No records found.")
    return df

def query_resumed_embryoscope_timeline(conn, prontuario):
    """Query resumed embryoscope timeline data for a specific prontuario (most recent record only)"""
    print(f"\n=== RESUMED EMBRYOSCOPE TIMELINE (MOST RECENT RECORD) ===")
    query = f"""
    SELECT * FROM gold.resumed_embryoscope_timeline 
    WHERE prontuario = {prontuario}
    ORDER BY period_month DESC
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in resumed_embryoscope_timeline")
    # if not df.empty:
    #     display(df)
    # else:
    #     print("No records found.")
    return df

def query_embryo_treatment (conn, prontuario):
    """Query embryoscope treatment data for a specific prontuario"""
    print(f"\n=== EMBRYOSCOPE TREATMENT ===")
    query = f"""
    SELECT 
        patient_PatientIDx
        ,	patient_PatientID
        ,	patient_FirstName	
        , patient_LastName	
        , patient_unit_huntington	
        , embryo_KIDDate
        , embryo_FertilizationTime 
        , treatment_TreatmentName	
        , patient_unit_huntington 
    FROM gold.embryoscope_embrioes 
    WHERE prontuario = {prontuario}
    ORDER BY treatment_TreatmentName DESC
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in embryoscope_embrioes")
    # if not df.empty:
    #     display(df)
    # else:
    #     print("No records found.")
    return df

def query_embryo_freeze_timeline(conn, prontuario):
    """Query embryo freeze timeline data for a specific prontuario"""
    print(f"\n=== EMBRYO FREEZE TIMELINE ===")
    query = f"""
    SELECT * FROM gold.embryo_freeze_timeline 
    WHERE prontuario = {prontuario}
    ORDER BY period_month DESC
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in embryo_freeze_timeline")
    # if not df.empty:
    #     display(df)
    # else:
    #     print("No records found.")
    return df

def query_embryo_billing_timeline(conn, prontuario):
    """Query embryo billing timeline data for a specific prontuario"""
    print(f"\n=== EMBRYO BILLING TIMELINE ===")
    query = f"""
    SELECT * FROM gold.embryo_billing_timeline 
    WHERE prontuario = {prontuario}
    ORDER BY period_month DESC
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in embryo_billing_timeline")
    # if not df.empty:
    #     display(df)
    # else:
    #     print("No records found.")
    return df

def query_comprehensive_embryo_timeline(conn, prontuario):
    """Query comprehensive embryo timeline data for a specific prontuario"""
    print(f"\n=== COMPREHENSIVE EMBRYO FREEZE TIMELINE ===")
    query = f"""
    SELECT * FROM gold.comprehensive_embryo_timeline 
    WHERE prontuario = {prontuario}
    ORDER BY period_month DESC
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in comprehensive_embryo_timeline")
    # if not df.empty:
    #     display(df)
    # else:
    #     print("No records found.")
    return df

def query_cryopreservation_events_timeline(conn, prontuario):
    """Query cryopreservation events timeline data for a specific prontuario"""
    print(f"\n=== CRYOPRESERVATION EVENTS TIMELINE ===")
    query = f"""
    SELECT * FROM gold.cryopreservation_events_timeline 
    WHERE prontuario = {prontuario}
    ORDER BY period_month DESC
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in cryopreservation_events_timeline")
    # if not df.empty:
    #     display(df)
    # else:
    #     print("No records found.")
    return df

def query_resumed_cryopreservation_events_timeline(conn, prontuario):
    """Query cryopreservation events timeline data for a specific prontuario"""
    print(f"\n=== CRYOPRESERVATION EVENTS TIMELINE ===")
    query = f"""
    SELECT * FROM gold.resumed_cryopreservation_events_timeline 
    WHERE prontuario = {prontuario}
    ORDER BY period_month DESC
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in resumed_cryopreservation_events_timeline")
    # if not df.empty:
    #     display(df)
    # else:
    #     print("No records found.")
    return df

def query_consultas_timeline(conn, prontuario):
    """Query consultation timeline data for a specific prontuario"""
    print(f"\n=== CONSULTAS TIMELINE ===")
    query = f"""
    SELECT * FROM gold.consultas_timeline 
    WHERE prontuario = {prontuario}
    ORDER BY period_month DESC
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in consultas_timeline")
    # if not df.empty:
    #     display(df)
    # else:
    #     print("No records found.")
    return df

def query_resumed_consultas_timeline(conn, prontuario):
    """Query resumed consultation timeline data for a specific prontuario (most recent record only)"""
    print(f"\n=== RESUMED CONSULTAS TIMELINE (MOST RECENT RECORD) ===")
    query = f"""
    SELECT * FROM gold.resumed_consultas_timeline 
    WHERE prontuario = {prontuario}
    ORDER BY period_month DESC
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in resumed_consultas_timeline")
    # if not df.empty:
    #     display(df)
    # else:
    #     print("No records found.")
    return df


def get_finops_data(conn, prontuario):
    """Get finops data for a specific patient"""
    print(f"\n=== RESUMED TRATAMENTOS TIMELINE (MOST RECENT RECORD) ===")

    query = f"""
        SELECT * FROM gold.finops_summary
        WHERE prontuario = {prontuario}
    """

    df = conn.execute(query).df()
    print(f"Found {len(df)} records in resumed_consultas_timeline")
    # if not df.empty:
    #     display(df)
    # else:
    #     print("No records found.")
    return df


In [9]:
# Updated query function with biopsy PGT-A timeline and cryopreservation data
def query_patient_data_with_timeline(prontuario):
    """
    Query all patient data including timeline data from gold layer tables.
    
    Args:
        prontuario (str or int): The patient prontuario number
        
    Returns:
        dict: Dictionary containing all DataFrames with results including timeline data
    """
    print(f"\n{'='*80}")
    print(f"QUERYING DATA FOR PRONTUARIO: {prontuario}")
    print(f"Query executed at: {datetime.now()}")
    print(f"{'='*80}")
    
    # Connect to databases
    clinisys_con = get_clinisys_connection()
    huntington_con = get_huntington_connection()
    
    try:
        # Query clinisys tables
        df_micromanipulacao = query_micromanipulacao(clinisys_con, prontuario)
        df_oocitos = query_micromanipulacao_oocitos(clinisys_con, prontuario)
        df_tratamentos = query_tratamentos(clinisys_con, prontuario)
        df_embrioes = query_embrioes_congelados(clinisys_con, prontuario)
        df_congelamentos_embrioes = query_congelamentos_embrioes(clinisys_con, prontuario)
        df_congelamentos_ovulos = query_congelamentos_ovulos(clinisys_con, prontuario)
        df_congelamentos_semen = query_congelamentos_semen(clinisys_con, prontuario)
        
        # Query mesclada table
        df_mesclada = query_mesclada_vendas(huntington_con, prontuario)

        # query Tratamentos timeline
        df_tratamentos_timeline = get_finops_data(huntington_con, prontuario)
        # df_billing = query_billing_timeline(huntington_con, prontuario)
        df_comprehensive = query_comprehensive_timeline(huntington_con, prontuario)
        df_resumed = query_resumed_timeline(huntington_con, prontuario)
        
        
        # Query biopsy PGT-A timeline
        df_biopsy_pgta = query_biopsy_pgta_timeline(huntington_con, prontuario)
        
        
        
        # Query embryoscope timeline
        df_embryoscope_timeline = query_embryoscope_timeline(huntington_con, prontuario)
        df_resumed_embryoscope = query_resumed_embryoscope_timeline(huntington_con, prontuario)
        
        # Query embryoscope usage
        df_embryoscope = query_embryo_treatment(huntington_con, prontuario)
        
        # Query cryopreservation timeline data
        df_embryo_freeze_timeline = query_embryo_freeze_timeline(huntington_con, prontuario)
        df_embryo_billing_timeline = query_embryo_billing_timeline(huntington_con, prontuario)
        df_comprehensive_embryo_timeline = query_comprehensive_embryo_timeline(huntington_con, prontuario)
        
        
        df_cryopreservation_events_timeline = query_cryopreservation_events_timeline(huntington_con, prontuario)
        df_resumed_cryopreservation_events_timeline = query_resumed_cryopreservation_events_timeline(huntington_con, prontuario)

        # Query consultation timeline data
        df_consultas_timeline = query_consultas_timeline(huntington_con, prontuario)
        df_resumed_consultas_timeline = query_resumed_consultas_timeline(huntington_con, prontuario)
        
        # Summary
        print(f"\n{'='*80}")
        print(f"QUERY SUMMARY FOR PRONTUARIO {prontuario}")
        print(f"{'='*80}")
        print(f"- view_micromanipulacao: {len(df_micromanipulacao)} records")
        print(f"- view_micromanipulacao_oocitos: {len(df_oocitos)} records")
        print(f"- view_tratamentos: {len(df_tratamentos)} records")
        print(f"- view_embrioes_congelados: {len(df_embrioes)} records")
        print(f"- view_congelamentos_embrioes: {len(df_congelamentos_embrioes)} records")
        print(f"- view_congelamentos_ovulos: {len(df_congelamentos_ovulos)} records")
        print(f"- view_congelamentos_semen: {len(df_congelamentos_semen)} records")
        print(f"- silver.mesclada_vendas: {len(df_mesclada)} records")
        print(f"- tratamentos_timeline: {len(df_tratamentos_timeline)} records")
        print(f"- biopsy_pgta_timeline: {len(df_biopsy_pgta)} records")
        # print(f"- billing_timeline: {len(df_billing)} records")
        print(f"- comprehensive_timeline: {len(df_comprehensive)} records")
        print(f"- resumed_timeline: {len(df_resumed)} records")
        print(f"- embryoscope_timeline: {len(df_embryoscope_timeline)} records")
        print(f"- resumed_embryoscope_timeline: {len(df_resumed_embryoscope)} records")
        print(f"- embryoscope_treatment: {len(df_embryoscope)} records")
        print(f"- embryo_freeze_timeline: {len(df_embryo_freeze_timeline)} records")
        print(f"- embryo_billing_timeline: {len(df_embryo_billing_timeline)} records")
        print(f"- comprehensive_embryo_timeline: {len(df_comprehensive_embryo_timeline)} records")
        print(f"- cryopreservation_events_timeline: {len(df_cryopreservation_events_timeline)} records")
        print(f"- resumed_cryopreservation_events_timeline: {len(df_resumed_cryopreservation_events_timeline)} records")
        print(f"- consultas_timeline: {len(df_consultas_timeline)} records")
        print(f"- resumed_consultas_timeline: {len(df_resumed_consultas_timeline)} records")
        print(f"{'='*80}")
        
        # Return results as dictionary
        results = {
            'micromanipulacao': df_micromanipulacao,
            'micromanipulacao_oocitos': df_oocitos,
            'tratamentos': df_tratamentos,
            'embrioes_congelados': df_embrioes,
            'congelamentos_embrioes': df_congelamentos_embrioes,
            'congelamentos_ovulos': df_congelamentos_ovulos,
            'congelamentos_semen': df_congelamentos_semen,
            'mesclada_vendas': df_mesclada,
            'tratamentos_timeline': df_tratamentos_timeline,
            'biopsy_pgta_timeline': df_biopsy_pgta,
            # 'billing_timeline': df_billing,
            'comprehensive_timeline': df_comprehensive,
            'resumed_timeline': df_resumed,
            'embryoscope_timeline': df_embryoscope_timeline,
            'resumed_embryoscope_timeline': df_resumed_embryoscope,
            'embryoscope_treatment': df_embryoscope,
            'embryo_freeze_timeline': df_embryo_freeze_timeline,
            'embryo_billing_timeline': df_embryo_billing_timeline,
            'comprehensive_embryo_timeline': df_comprehensive_embryo_timeline,
            'cryopreservation_events_timeline': df_cryopreservation_events_timeline,
            'resumed_cryopreservation_events_timeline': df_resumed_cryopreservation_events_timeline,
            'consultas_timeline': df_consultas_timeline,
            'resumed_consultas_timeline': df_resumed_consultas_timeline
        }
        
        return results
        
    finally:
        # Close connections
        clinisys_con.close()
        huntington_con.close()
        print("\nDatabase connections closed.")


## Query Finops Silver Layer Table


In [10]:
## Example usage with comprehensive timeline data including cryopreservation


prontuario = '788592'
prontuario = 836188


# Query all patient data including comprehensive timeline and cryopreservation data
results_with_timeline = query_patient_data_with_timeline(prontuario)


QUERYING DATA FOR PRONTUARIO: 836188
Query executed at: 2025-11-04 10:41:58.710821

=== VIEW_MICROMANIPULACAO ===
Found 4 records in view_micromanipulacao

=== VIEW_MICROMANIPULACAO_OOCITOS ===
Found 37 records in view_micromanipulacao_oocitos

=== VIEW_TRATAMENTOS ===
Found 5 records in view_tratamentos

=== VIEW_EMBRIOES_CONGELADOS ===
Found 2 records in view_embrioes_congelados

=== VIEW_CONGELAMENTOS_EMBRIOES ===
Found 2 records in view_congelamentos_embrioes

=== VIEW_CONGELAMENTOS_OVULOS ===
Found 0 records in view_congelamentos_ovulos

=== VIEW_CONGELAMENTOS_SEMEN ===
Found 0 records in view_congelamentos_semen

=== SILVER.MESCLADA_VENDAS ===
Found 53 records in silver.mesclada_vendas

=== RESUMED TRATAMENTOS TIMELINE (MOST RECENT RECORD) ===
Found 1 records in resumed_consultas_timeline

=== COMPREHENSIVE BIOPSY PGT-A TIMELINE TIMELINE ===
Found 3 records in comprehensive_biopsy_pgta_timeline

=== RESUMED BIOPSY PGT-A TIMELINE TIMELINE (MOST RECENT RECORD) ===
Found 1 records 

# 0. Pagamentos

In [11]:
# Billing data
print("=== BILLING DATA ===")
mesclada_data = results_with_timeline['mesclada_vendas']
print(f"Records: {len(mesclada_data)}")
if not mesclada_data.empty:
    display(mesclada_data)
else:
    print("No billing data found.")


=== BILLING DATA ===
Records: 53


,Cliente,Nome,Paciente,Nom Paciente,prontuario,DT Emissao,Descricao,Qntd.,Total,Descrição Gerencial,Loja,Tipo da nota,Numero,Serie Docto.,NF Eletr.,Vend. 1,Médico,Cliente_totvs,Operador,Produto,Valor Mercadoria,Custo,Custo Unit,Desconto,Unidade,Mês,Ano,Cta-Ctbl,Cta-Ctbl Eugin,Interno/Externo,Descrição Mapping Actividad,Ciclos,Qnt Cons.,Grp,Descr.TES,Lead Time,Data do Ciclo,Fez Ciclo?,line_number,extraction_timestamp,file_name
0,836188,CARLA TAINA SO,836188,CARLA TAINA SOARES MOURAO,836188,2025-10-09,MENOPUR MD 600UI (C.E.),1.0,1512.00,Estimulação,10150,502,36619,RPS,36472,10,EDUARDO LEME ALVES MOTTA,878870,,000000000004709,0.0,0.0,0.0,0.00,Ibirapuera,0,0,11310100005,,,Estimulação,0,0,1,PRESTACAO DE SERVICO,None,NaT,False,4969,20251023_201129,Mescladas2 (4).xlsx
1,836189,HERALDO FALCAO,836188,CARLA TAINA SOARES MOURAO,836188,2025-09-09,MENOPUR 75UI 1 FA DILUENTE IML + 5 FRASC,1.0,1025.00,Estimulação,10150,510,31119,RPS,31002,10,EDUARDO LEME ALVES MOTTA,878871,,000000000005926,0.0,0.0,0.0,0.00,Ibirapuera,0,0,11310100005,,,Estimulação,0,0,1,PRESTACAO DE SERVICO,None,NaT,False,19096,20251023_201129,Mescladas2 (4).xlsx
2,836189,HERALDO FALCAO,836188,CARLA TAINA SOARES MOURAO,836188,2025-09-09,"OVIDREL FPEN 250MCG 0,5 ML",1.0,609.00,Estimulação,10150,510,31119,RPS,31002,10,EDUARDO LEME ALVES MOTTA,878871,,000000000004644,0.0,0.0,0.0,0.00,Ibirapuera,0,0,11310100005,,,Estimulação,0,0,1,PRESTACAO DE SERVICO,None,NaT,False,19097,20251023_201129,Mescladas2 (4).xlsx
3,836188,CARLA TAINA SO,836188,CARLA TAINA SOARES MOURAO,836188,2025-08-28,MENOPUR MD 600UI (C.E.),1.0,1512.00,Estimulação,10150,510,29450,RPS,29334,10,EDUARDO LEME ALVES MOTTA,878870,,000000000004709,0.0,0.0,0.0,0.00,Ibirapuera,0,0,11310100005,,,Estimulação,0,0,1,PRESTACAO DE SERVICO,None,NaT,False,424455,20251023_201129,Mescladas2 (4).xlsx
4,836189,HERALDO FALCAO,836188,CARLA TAINA SOARES MOURAO,836188,2025-08-28,FIV - MEDICOS INTERNOS,1.0,24150.78,FIV,10150,502,29467,RPS,29323,10,EDUARDO LEME ALVES MOTTA,878871,,S3551,0.0,0.0,0.0,6423.22,Ibirapuera,0,0,31110100001,,,FIV,1,0,1,PRESTACAO DE SERVICO,None,NaT,False,424483,20251023_201129,Mescladas2 (4).xlsx
5,836189,HERALDO FALCAO,836188,CARLA TAINA SOARES MOURAO,836188,2025-08-28,ZYMOT,1.0,954.00,Zymot,10150,502,29467,RPS,29323,10,EDUARDO LEME ALVES MOTTA,878871,,S3573.04,0.0,0.0,0.0,0.00,Ibirapuera,0,0,31110100017,,,Zymot,0,0,1,PRESTACAO DE SERVICO,None,NaT,False,424484,20251023_201129,Mescladas2 (4).xlsx
6,836188,CARLA TAINA SO,836188,CARLA TAINA SOARES MOURAO,836188,2025-08-27,MENOPUR MD 600UI (C.E.),1.0,1512.00,Estimulação,10150,510,29306,RPS,29340,10,EDUARDO LEME ALVES MOTTA,878870,,000000000004709,0.0,0.0,0.0,0.00,Ibirapuera,0,0,11310100005,,,Estimulação,0,0,1,PRESTACAO DE SERVICO,None,NaT,False,424215,20251023_201129,Mescladas2 (4).xlsx
7,836188,CARLA TAINA SO,836188,CARLA TAINA SOARES MOURAO,836188,2025-08-27,MERIONAL 75UI,1.0,252.00,Estimulação,10150,510,29343,RPS,29213,10,EDUARDO LEME ALVES MOTTA,878870,,000000000006818,0.0,0.0,0.0,0.00,Ibirapuera,0,0,11310100005,,,Estimulação,0,0,1,PRESTACAO DE SERVICO,None,NaT,False,424273,20251023_201129,Mescladas2 (4).xlsx
8,836189,HERALDO FALCAO,836188,CARLA TAINA SOARES MOURAO,836188,2025-08-16,GENOTROPIN 12 MG,1.0,1230.00,Estimulação,10150,502,26902,RPS,26792,10,EDUARDO LEME ALVES MOTTA,878871,,000000000008429,0.0,0.0,0.0,0.00,Ibirapuera,0,0,11310100005,,,Estimulação,0,0,1,PRESTACAO DE SERVICO,None,NaT,False,421550,20251023_201129,Mescladas2 (4).xlsx
9,836189,HERALDO FALCAO,836188,CARLA TAINA SOARES MOURAO,836188,2025-08-16,MENOPUR MD 1200UI (C.E.),2.0,6048.00,Estimulação,10150,510,26902,RPS,26792,10,EDUARDO LEME ALVES MOTTA,878871,,000000000004711,0.0,0.0,0.0,0.00,Ibirapuera,0,0,11310100005,,,Estimulação,0,0,1,PRESTACAO DE SERVICO,None,NaT,False,421551,20251023_201129,Mescladas2 (4).xlsx


# 1. Tratamentos

In [12]:
# Treatments timeline data
print("=== TREATMENTS TIMELINE DATA ===")
tratamentos_timeline_data = results_with_timeline['tratamentos_timeline']
print(f"Records: {len(tratamentos_timeline_data)}")
if not tratamentos_timeline_data.empty:
    display(tratamentos_timeline_data)
else:
    print("No treatments timeline data found.")


=== TREATMENTS TIMELINE DATA ===
Records: 1


,prontuario,cycle_with_transfer,cycle_without_transfer,cycle_total,treatment_paid_count,treatment_paid_total,timeline_first_date,timeline_last_date,billing_first_date,billing_last_date,prontuario_genitores,medico_nome,flag_is_donor,flag_cycles_no_payments,flag_more_cycles_than_payments,flag_no_cycles_but_payments,flag_less_cycles_than_payments,timeline_unidade
0,836188.0,1,4,5,4,101365.68,2024-08-06,2025-09-01,2024-07-29,2025-08-28,836188.0,Eduardo Leme Alves da Motta,0,0,1,0,0,1. HTT SP - Ibirapuera


In [13]:
# Treatments data
print("=== TREATMENTS DATA ===")
tratamentos_data = results_with_timeline['tratamentos']
print(f"Records: {len(tratamentos_data)}")
if not tratamentos_data.empty:
    display(tratamentos_data)
else:
    print("No treatments data found.")


=== TREATMENTS DATA ===
Records: 5


,id,prontuario,unidade,idade_esposa,idade_marido,paciente_tratamento,tentativa,data_procedimento,hora_procedimento,tipo_procedimento,motivo_congelamento,tipo_inducao,protocolo,tipo_ciclo_congelado,tipo_preparo_endometrial,num_congelados,num_transferir,origem_ovulo,status_ovulo,prontuario_doadora,origem_espermatozoide,fonte_espermatozoide,status_espermatozoide,origem_material,material_heterologo,utero_substituicao,prontuario_genitores,prontuario_uter_substituicao,doacao_ovulos,destino_doacao_ovulos,prontuario_receptora,complemento_receptora,doacao_embrioes,era,emma,alice,matrice_lab,imunohistoquimica,outros_biopsia,consentimento,financeiro,embryoscope,pgt,era_auxiliar,recomendacao_transferencia,resultado_era,emma_resultado,alice_resultado,picsi,hatching_od,hatching_oe,medicacao_parental,peso_paciente,altura_paciente,fumante_paciente,data_dum,peso_conjuge,altura_conjuge,fumante_conjuge,fator_infertilidade1,fator_infertilidade2,fator_infertilidade3,fsh_exame,foliculos_antrais,data_histeroscopia,data_inicio_inducao,dias_inducao_preparo,paciente_inducao,ropa,data_inicio_agonista,gnrh,fsh_agonista,lh_agonista,outros_medicamentos,gonal,menogon,menopur,pergoveris,elonva,serofene_clomid_indux,luveris,puregon,letrozole,fostimon,merional,rekovelle,descricao_outras_medicacoes,dose_total_fsh,dose_total_lh,dose_total_fsh_lh,coorte_e2,coorte_aco,coorte_progesterona,coq10,hormonal_combinado,coorte_testosterona,agonista_coorte,coorte_antagonista,coorte_dhea,utilizado_hcg,motivo_cancelamento_hcg,data_hcg,horario_planejado_hcg,horario_real_hcg,ovidrel,gonapeptyl,dose_gonapeptyl,outras_complicacoes_hcg,dose_outras_complicacoes,lupron,outros_hcg,descricao_outros_hcg,data_inicio_progesterona,p4_inicio_progesterona,estradiol_fase_lutea,progesterona_fase_lutea,progesterona_oral,progesterona_vaginal,progesterona_injetavel,combinacao_oral_vaginal,nenhuma_progesterona,hcg_progesterona,data_congelamento,metodo_descongelamento,valor_palheta,manter_excedentes,observacoes_descongelamento,dia_transferencia,data_transferencia,medida_endometrio,padrao_endometrio,p4_transferencia,motivo_nao_transferir,motivo_cancelamento_tratamento,ligacao_enfermagem,beta_valor,beta_valor2,data_ultrassom1,data_ultrassom2,num_sacos_gestacionais,tsh,data_ultrassom1_contato,descricao_ultrassom1,bcf_embrião1_status,bcf_embrião1_fonte,data_ultrassom2_contato,descricao_ultrassom2,data_ultrassom3_contato,descricao_ultrassom3,resultado_tratamento,analise_genetica,resultado_analise_genetica,motivo_perda_aborto,data_nascimento_bebes,obstetra,nome_bebe1,sexo_bebe1,peso_bebe1,altura_bebe1,idade_gestacional_bebe1,via_parto_bebe1,viabilidade_bebe1,anomalias_bebe1,nome_bebe2,sexo_bebe2,peso_bebe2,altura_bebe2,idade_gestacional_bebe2,via_parto_bebe2,viabilidade_bebe2,anomalias_bebe2,nome_bebe3,sexo_bebe3,peso_bebe3,altura_bebe3,idade_gestacional_bebe3,via_parto_bebe3,viabilidade_bebe3,anomalias_bebe3,nome_bebe4,sexo_bebe4,peso_bebe4,altura_bebe4,idade_gestacional_bebe4,via_parto_bebe4,viabilidade_bebe4,anomalias_bebe4,observacoes_bebes,observacoes_gerais,usuario_responsavel,responsavel_informacoes,hash,extraction_timestamp
0,36980,836188,1,35,36,esposa,5,2025-09-01,None,Ciclo a Fresco FIV,None,Convencional,None,None,Hormonal,None,None,Homólogo,Fresco,NaN,Homólogo,Sêmen,Fresco,None,None,Não,NaN,NaN,Não,None,NaN,None,Não,None,None,None,None,None,None,None,None,None,PGT-A,None,None,None,None,None,Não,Não,Sim,Não,"58,00","1,56",None,NaT,None,None,None,Fator feminino anatômico,None,None,None,None,None,2025-08-16,None,esposa,None,Não,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,Não,None,None,None,None,None,Não utilizado,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,No transfer,None,None,None,None,Ausência de desenvolvimento embrionário,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,No transfer,Não,None,None,Non

In [14]:
# Billing data
print("=== BILLING DATA TREATMENTS===")

condicao = mesclada_data['Descrição Gerencial'].isin([
    'Coleta - Adicional', 
    'Coleta - Crio', 
    'FET / FOT', 
    'Pró-Fiv', 
    'FIV', 
    'Inseminação', 
    'Ovodoação', 
    'Embriodoação'
])

print(f"Records: {len(mesclada_data[condicao])}")
mesclada_data[condicao]

=== BILLING DATA TREATMENTS===
Records: 4


,Cliente,Nome,Paciente,Nom Paciente,prontuario,DT Emissao,Descricao,Qntd.,Total,Descrição Gerencial,Loja,Tipo da nota,Numero,Serie Docto.,NF Eletr.,Vend. 1,Médico,Cliente_totvs,Operador,Produto,Valor Mercadoria,Custo,Custo Unit,Desconto,Unidade,Mês,Ano,Cta-Ctbl,Cta-Ctbl Eugin,Interno/Externo,Descrição Mapping Actividad,Ciclos,Qnt Cons.,Grp,Descr.TES,Lead Time,Data do Ciclo,Fez Ciclo?,line_number,extraction_timestamp,file_name
4,836189,HERALDO FALCAO,836188,CARLA TAINA SOARES MOURAO,836188,2025-08-28,FIV - MEDICOS INTERNOS,1.0,24150.78,FIV,10150,502,29467,RPS,29323,10,EDUARDO LEME ALVES MOTTA,878871,,S3551,0.0,0.0,0.0,6423.22,Ibirapuera,0,0,31110100001,,,FIV,1,0,1,PRESTACAO DE SERVICO,None,NaT,False,424483,20251023_201129,Mescladas2 (4).xlsx
14,836189,HERALDO FALCAO,836188,CARLA TAINA SOARES MOURAO,836188,2025-04-29,FIV - MEDICOS INTERNOS,1.0,25987.90,FIV,10150,502,8507,RPS,8475,10,EDUARDO LEME ALVES MOTTA,878871,,S3551,0.0,0.0,0.0,4586.10,Ibirapuera,0,0,31110100001,,,FIV,1,0,1,PRESTACAO DE SERVICO,None,NaT,False,400726,20251023_201129,Mescladas2 (4).xlsx
33,836189,HERALDO FALCAO,836188,CARLA TAINA SOARES MOURAO,836188,2024-10-25,COLETA ADICIONAL - MEDICOS INTERNOS,1.0,21937.00,Coleta - Adicional,10101,502,224534,RPS,223141,10,EDUARDO LEME ALVES MOTTA,878871,,S3547.04,0.0,0.0,0.0,1357.00,Ibirapuera,0,0,31110100001,,,Coleta - Adicional,1,0,1,PRESTACAO DE SERVICO,None,NaT,False,322636,20251023_201129,Mescladas2 (4).xlsx
48,836188,CARLA TAINA SO,836188,CARLA TAINA SOARES MOURAO,836188,2024-07-29,FIV - MEDICOS INTERNOS,1.0,29290.00,FIV,10101,502,210892,RPS,209539,10,EDUARDO LEME ALVES MOTTA,878870,,S3551,0.0,0.0,0.0,0.00,Ibirapuera,0,0,31110100001,,,FIV,1,0,1,PRESTACAO DE SERVICO,None,NaT,False,307453,20251023_201129,Mescladas2 (4).xlsx


# 2. Biópsia - PGT-A

In [15]:
# Comprehensive timeline data
print("=== COMPREHENSIVE TIMELINE DATA ===")
comprehensive_timeline_data = results_with_timeline['comprehensive_timeline']
print(f"Records: {len(comprehensive_timeline_data)}")
if not comprehensive_timeline_data.empty:
    display(comprehensive_timeline_data)
else:
    print("No comprehensive timeline data found.")


=== COMPREHENSIVE TIMELINE DATA ===
Records: 3


,prontuario,period_month,biopsy_count,cumulative_biopsy_count,biopsy_payment_amount,biopsy_payment_count,biopsy_payment_qtd,cumulative_biopsy_payment_qtd,biopsy_missing_payment,pgta_test_count,cumulative_pgta_test_count,pgta_payment_amount,pgta_payment_count,pgta_payment_qtd,cumulative_pgta_payment_qtd,pgta_missing_payment
0,836188,2025-05,0,2.0,1880.0,1,1.0,2.0,0.0,0,2.0,1100.0,1,1.0,2.0,0.0
1,836188,2025-04,1,2.0,0.0,0,0.0,1.0,1.0,1,2.0,0.0,0,0.0,1.0,1.0
2,836188,2024-10,1,1.0,1880.0,1,1.0,1.0,0.0,1,1.0,1100.0,1,1.0,1.0,0.0


In [16]:
# Resumed timeline data
print("=== RESUMED TIMELINE DATA ===")
resumed_timeline_data = results_with_timeline['resumed_timeline']
print(f"Records: {len(resumed_timeline_data)}")
if not resumed_timeline_data.empty:
    display(resumed_timeline_data)
else:
    print("No resumed timeline data found.")


=== RESUMED TIMELINE DATA ===
Records: 1


,prontuario,period_month,biopsy_count,cumulative_biopsy_count,biopsy_payment_amount,biopsy_payment_count,biopsy_payment_qtd,cumulative_biopsy_payment_qtd,biopsy_missing_payment,pgta_test_count,cumulative_pgta_test_count,pgta_payment_amount,pgta_payment_count,pgta_payment_qtd,cumulative_pgta_payment_qtd,pgta_missing_payment
0,836188,2025-05,0,2.0,1880.0,1,1.0,2.0,0.0,0,2.0,1100.0,1,1.0,2.0,0.0


In [17]:
# Billing data
print("=== BILLING DATA BIOPSY===")

condicao = mesclada_data['Descrição Gerencial'].isin([
    'Biópsia Embrionária',
    'PGT-A'
])

print(f"Records: {len(mesclada_data[condicao])}")
mesclada_data[condicao]

=== BILLING DATA BIOPSY===
Records: 4


,Cliente,Nome,Paciente,Nom Paciente,prontuario,DT Emissao,Descricao,Qntd.,Total,Descrição Gerencial,Loja,Tipo da nota,Numero,Serie Docto.,NF Eletr.,Vend. 1,Médico,Cliente_totvs,Operador,Produto,Valor Mercadoria,Custo,Custo Unit,Desconto,Unidade,Mês,Ano,Cta-Ctbl,Cta-Ctbl Eugin,Interno/Externo,Descrição Mapping Actividad,Ciclos,Qnt Cons.,Grp,Descr.TES,Lead Time,Data do Ciclo,Fez Ciclo?,line_number,extraction_timestamp,file_name
10,836189,HERALDO FALCAO,836188,CARLA TAINA SOARES MOURAO,836188,2025-05-02,BIOPSIA POR EMBRIOES - MEDICOS INTERNOS,1.0,1880.0,Biópsia Embrionária,10150,502,8595,RPS,8543,10,EDUARDO LEME ALVES MOTTA,878871,,S3563.09,0.0,0.0,0.0,0.0,Ibirapuera,0,0,31110100024,,,Biópsia - Embriões,0,0,1,PRESTACAO DE SERVICO,None,NaT,False,400841,20251023_201129,Mescladas2 (4).xlsx
11,836189,HERALDO FALCAO,836188,CARLA TAINA SOARES MOURAO,836188,2025-05-02,ANALISE DE EMBRIOES PGT-A,1.0,1100.0,PGT-A,10150,502,8595,RPS,8543,10,EDUARDO LEME ALVES MOTTA,878871,,S3563.30,0.0,0.0,0.0,0.0,Ibirapuera,0,0,31110100015,,,PGT-A,0,0,1,PRESTACAO DE SERVICO,None,NaT,False,400842,20251023_201129,Mescladas2 (4).xlsx
31,836188,CARLA TAINA SO,836188,CARLA TAINA SOARES MOURAO,836188,2024-10-31,BIOPSIA POR EMBRIOES - MEDICOS INTERNOS,1.0,1880.0,Biópsia Embrionária,10101,502,225025,RPS,223631,10,EDUARDO LEME ALVES MOTTA,878870,,S3563.09,0.0,0.0,0.0,0.0,Ibirapuera,0,0,31110100024,,,Biópsia - Embriões,0,0,1,PRESTACAO DE SERVICO,None,NaT,False,323251,20251023_201129,Mescladas2 (4).xlsx
32,836188,CARLA TAINA SO,836188,CARLA TAINA SOARES MOURAO,836188,2024-10-31,ANALISE DE EMBRIOES PGT-A,1.0,1100.0,PGT-A,10101,502,225025,RPS,223631,10,EDUARDO LEME ALVES MOTTA,878870,,S3563.30,0.0,0.0,0.0,0.0,Ibirapuera,0,0,31110100015,,,PGT-A,0,0,1,PRESTACAO DE SERVICO,None,NaT,False,323252,20251023_201129,Mescladas2 (4).xlsx


In [18]:
# Micromanipulation oocytes data
print("=== MICROMANIPULATION OOCYTES DATA ===")
oocitos_data = results_with_timeline['micromanipulacao_oocitos']
print(f"Records: {len(oocitos_data)}")
if not oocitos_data.empty:
    display(oocitos_data)
else:
    print("No micromanipulation oocytes data found.")


=== MICROMANIPULATION OOCYTES DATA ===
Records: 37


,id,id_micromanipulacao,diaseguinte,Maturidade,RC,ComentariosAntes,Embriologista,PI,TCD,AH,PGD,ResultadoPGD,IdentificacaoPGD,Fertilizacao,CorpusculoPolar,ComentariosDepois,GD1,OocitoDoado,ICSI,OrigemOocito,InseminacaoOocito,NClivou_D2,NCelulas_D2,Frag_D2,Blastomero_D2,NClivou_D3,NCelulas_D3,Frag_D3,Blastomero_D3,GD2,NClivou_D4,NCelulas_D4,Compactando_D4,MassaInterna_D4,Trofoblasto_D4,NClivou_D5,NCelulas_D5,Compactando_D5,MassaInterna_D5,Trofoblasto_D5,NClivou_D6,NCelulas_D6,Compactando_D6,MassaInterna_D6,Trofoblasto_D6,NClivou_D7,NCelulas_D7,Compactando_D7,MassaInterna_D7,Trofoblasto_D7,score_maia,relatorio_ia,hash,extraction_timestamp,flag_embryoscope,embryo_number
0,210336,21740,Não,MII,None,None,3652,None,Descartado,None,None,None,None,2PN,C1,Fresco,None,Não,None,Fresco,ICSI,None,None,None,None,Sim,3,1,Simétricos,None,None,None,None,None,None,Não,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,5321f4f5c8de355ef33e86111b480eae,2025-07-21 21:40:02,1,1
1,210337,21740,Não,MII,None,None,3652,None,Descartado,None,None,None,None,2CP,C1,Fresco,None,Não,None,Fresco,ICSI,None,None,None,None,Sim,4,3,Simétricos,None,None,None,None,None,None,Não,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,d3428f4f01179a16c246271ddbef0139,2025-07-21 21:40:02,1,2
2,210338,21740,Não,MII,None,None,3652,None,Descartado,None,None,None,None,2PN,C1,Fresco,None,Não,None,Fresco,ICSI,None,None,None,None,Sim,4,3,Simétricos,None,None,None,None,None,None,Sim,6,Fragmentação 3,None,None,Não,None,None,None,None,None,None,None,None,None,None,None,f536a9d96f935a7e78506911be5faef9,2025-07-21 21:40:02,1,3
3,210339,21740,Não,MII,None,None,3652,None,Descartado,None,None,None,None,2CP,None,Fresco,None,Não,None,Fresco,ICSI,None,None,None,None,Sim,2,2,Simétricos,None,None,None,None,None,None,Não,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,367157fc2a8c1ae0b19f6974e87529bc,2025-07-21 21:40:02,1,4
4,210340,21740,Não,MII,None,None,3652,None,Descartado,None,None,None,None,2PN,C1,Fresco,None,Não,None,Fresco,ICSI,None,None,None,None,Sim,4,3,Simétricos,None,None,None,None,None,None,Sim,6,Fragmentação 3,None,None,Não,None,None,None,None,None,None,None,None,None,None,None,fe07135145afad2eedc0f2d3e22aaaff,2025-07-21 21:40:02,1,5
5,210341,21740,Não,MII,None,None,3652,None,Descartado,None,None,None,None,1CP,C0,Fresco,None,Não,None,Fresco,ICSI,None,None,None,None,Não,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,77add358d8cfc43e3d8b300a24c5791e,2025-07-21 21:40:02,1,6
6,210342,21740,Não,MII,None,None,3652,None,Descartado,None,None,None,None,1CP,C0,Fresco,None,Não,None,Fresco,ICSI,None,None,None,None,Não,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,e32a1c2a61328f2ba83b19e919c5e359,2025-07-21 21:40:02,1,7
7,210343,21740,Não,MII,None,None,3652,None,Descartado,None,None,None,None,2PN,C0,Fresco,None,Não,None,Fresco,ICSI,None,None,None,None,Sim,4,3,Simétricos,None,None,None,None,None,None,Sim,5,Fragmentação 3,None,None,Não,None,None,None,None,None,None,None,None,None,None,None,4ef5ab2e03f8780d3d58efa0778827b8,2025-07-21 21:40:02,1,8
8,210344,21740,Não,MI,None,None,None,None,Descartado,None,None,None,None,None,C0,None,None,Não,None,Fresco,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,c4116f8684a23bb7d2ddadaa3a8928be,2025-07-21 21:40:02,0,<NA>
9,210345,21740,Não,MI,None,None,None,None,Descartado,None,None,None,None,None,C0,None,None,Não,None,Fresco,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,1ba0f02807e0585b8a02accb2a3fd87e,2025-07-21 21:40:02,0,<NA>


In [19]:
# Frozen embryos data
print("=== FROZEN EMBRYOS DATA ===")
embrioes_data = results_with_timeline['embrioes_congelados']
print(f"Records: {len(embrioes_data)}")
if not embrioes_data.empty:
    display(embrioes_data)
else:
    print("No frozen embryos data found.")


=== FROZEN EMBRYOS DATA ===
Records: 2


,id,id_oocito,id_congelamento,id_descongelamento,prontuario,pailletes,pailletes_id,cores,embriao,doado,lote,celulas,qualidade,qualidade_recongelamento,comentarios,comentarios_descongelamento,ah,dia2,maturidade,dia2_2,dia2_3,dia3,dia3_2,dia3_3,dia4,dia4_2,dia4_3,dia4_4,gd,gd2,gd3,dia5,dia5_2,dia5_3,dia5_4,dia6,dia6_2,dia6_3,dia6_4,dia7,dia7_2,dia7_3,dia7_4,clivou_d2,clivou_d3,clivou_d4,clivou_d5,clivou_d6,clivou_d7,transferidos,pgd,resultado_pgd,colapso,pgd_congelamento,resultado_pgd_congelamento,mito_teste_pgd_congelamento,dia_congelamento,score_maia,tanque_amostra,caneca_amostra,rack_amostra,observacao,destino,hash,extraction_timestamp
0,84422,250981,28623,0,836188,P3,E22430,<NA>,#3,None,None,None,Blastocisto Grau 5 - C - C,<NA>,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,<NA>,None,None,None,None,None,Descartado,None,None,"XY, Trissomia chr 10 Masculino, Aneuploide",Sim,Aneuploide,2.0,D7,1.27,AM,2,E22430,None,<NA>,c50c9fb507432a2694ba4f8775ec023f,2025-07-21 21:38:57
1,78598,223734,27090,16698,836188,P3,E21680,<NA>,#3,None,None,None,Blastocisto Grau 4 - B - B,<NA>,None,#3 Euploide,Não,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,Blastocisto Grau 4,B,B,None,None,None,None,<NA>,None,None,None,None,None,Transferido,Sim,Euploide,"MASCULINO, EUPLOIDE",Sim,Euploide,4.0,D6,1.58,AK,1,E21680,None,<NA>,47ea7965f56f82c7d7cebc71e5e17489,2025-07-21 21:38:57


In [20]:
# Display each result in separate cells

# Micromanipulation data
print("=== MICROMANIPULATION DATA ===")
micromanipulacao_data = results_with_timeline['micromanipulacao']
print(f"Records: {len(micromanipulacao_data)}")
if not micromanipulacao_data.empty:
    display(micromanipulacao_data)
else:
    print("No micromanipulation data found.")


=== MICROMANIPULATION DATA ===
Records: 4


,codigo_ficha,numero_caso,prontuario,IdadeEsposa_DG,IdadeMarido_DG,Data_DL,codigo_congelamento_semen,horario_inicial_fert,horario_final_fert,Aspiracao_DL,medico_id,responsavel_puncao_id,abstinencia_recomendada,recepcao_ovulos,CicloDoadora,IdadeDoadora,checagem_d4,Denud_DL,checagem_fertilizacao,checagem_d2,checagem_d3,checagem_d5,checagem_d6,checagem_d7,foliculos,oocitos,sangue_cateter_interno,dificuldade_transf,retorno_embrioes,vezes_retorno,sangue_cateter_externo,ICSIDescongelados,CicloDescongelamento,pgd_oocito,pgd_d3,pgd_d5,data_procedimento,tecnica_injetados,motilidade,morfologia_injetados,gradiente_descontinuo,mini_gradiente,data_procedimento_2,tecnica_injetados_2,motilidade_2,morfologia_injetados_2,unidade_id,data_microtese,data_microtese_2,validade7,responsavel_denud_id,profissional_fertilizacao_id,profissional_d2_id,profissional_d3_id,profissional_d5_id,profissional_d6_id,profissional_d7_id,validade11,materiais_insumos,local_coleta,fonte_outros,gradiente_descontinuo_2,viscosidade,volume_amostra_utilizada,morfologia_pre,morfologia_pos,zymot,mini_gradiente_2,local_coleta_2,fonte_outros_2,observacoes_resumo_ciclos,viscosidade_2,volume_amostra_utilizada_2,morfologia_pre_2,morfologia_pos_2,zymot_2,validade20,gradiente,lote21,validade21,agulha_icsi,lote_agulha_icsi,validade_agulha_icsi,agulha_aspir,lote_agulha_aspir,validade_agulha_aspir,incubadora,abstinencia_dias,meios,responsavel_oocitos_id,fonte,inicio_ps,termino_ps,volume_ps1,CongelamentoOvulos,concentracao_progressivos_pre,c_ps1,d_ps1,conc_ps1,diametro_minimo_pse,diametro_medio_pse,diametro_maximo_pse,tubulos_avaliados_pse,peso_total_pse,peso_microtese_pse,peso_biopsia_pse,diametro_minimo_psd,diametro_medio_psd,diametro_maximo_psd,tubulos_avaliados_psd,peso_total_psd,peso_microtese_psd,peso_biopsia_psd,responsavel_transfer_id,MorfologiaAlterada,EstadoSptz,CongelamentoSemenHom,VialsDescongeladasHom,CongelamentoSemenHet,VialsDescongeladasHet,check_gradiente_ps,concentrado,check_swimup,check_lavagem,volume_ps2,MotivoCongelamentoOvulos,concentracao_progressivos_pos,c_ps2,d_ps2,conc_ps2,comentarios_ps,responsavel_ps,fonte2,inicio_ps2,termino_ps2,volume_ps3,concentracao_progressivos_pre_2,c_ps3,d_ps3,conc_ps3,diametro_minimo_pse2,diametro_medio_pse2,diametro_maximo_pse2,tubulos_avaliados_pse2,peso_total_pse2,peso_microtese_pse2,peso_biopsia_pse2,diametro_minimo_psd2,diametro_medio_psd2,diametro_maximo_psd2,tubulos_avaliados_psd2,peso_total_psd2,peso_microtese_psd2,peso_biopsia_psd2,timelapse,EstadoSptz2,codigo_congelamento_semen_2,CongelamentoSemenHom2,VialsDescongeladasHom2,CongelamentoSemenHet2,VialsDescongeladasHet2,check_gradiente_ps2,concentrado_2,check_swimup2,check_lavagem2,volume_ps4,concentracao_progressivos_pos_2,c_ps4,d_ps4,conc_ps4,comentarios_ps2,responsavel_ps2,transferidos,cateter,lote_cateter,validade_cateter,motivo_nao_transferencia,Transfer_D5,medico_responsavel_id,azoospermia,cod1_crio,cod2_crio,tanque,cane,tecnica,Observacoes,responsavel_labfiv,responsavel_labfiv_data,tipo_paciente,responsavel_biopsia,alteracoes_oocitarias_normal,alteracoes_oocitarias_granulacao,alteracoes_oocitarias_pelucida,alteracoes_oocitarias_oval,alteracoes_oocitarias_inclusoes,alteracoes_oocitarias_pervitelineo_aumentado,alteracoes_oocitarias_irregular,alteracoes_oocitarias_necrotica,alteracoes_oocitarias_pervitelineo_granular,alteracoes_oocitarias_ser,alteracoes_oocitarias_corpusculo_frag,alteracoes_oocitarias_vacuolos,alteracoes_oocitarias_corpusculo_aum,alteracoes_oocitarias_membrana,alteracoes_oocitarias_corpusculo_peq,alteracoes_oocitarias_citoplasma,alteracoes_oocitarias_corpusculo_deg,controle_anual,maia,hash,extraction_timestamp
0,27188,3021/25,836188,35,36,2025-09-01,None,12:10,12:20,09:05,10,3613,None,Não,None,None,None,12:00,08:00,None,08:00,08:00,None,None,7,7,None,None,None,None,None,Não,None,None,None,None,2025-09-01,ICSI,05 espermatozoides móveis progressivos\r\n,05 espermatozoides normais\r\n,None,None,None,None,None,None,0000000001,<NA>,<NA>,<NA>,4005,3492,None,400

# 3. Embryoscope

In [21]:
# Embryoscope timeline data
print("=== EMBRYOSCOPE TIMELINE DATA ===")
embryoscope_timeline_data = results_with_timeline['embryoscope_timeline']
print(f"Records: {len(embryoscope_timeline_data)}")
if not embryoscope_timeline_data.empty:
    display(embryoscope_timeline_data)
else:
    print("No embryoscope timeline data found.")


=== EMBRYOSCOPE TIMELINE DATA ===
Records: 4


,prontuario,period_month,embryoscope_usage_count,cumulative_embryoscope_usage,billing_events_count,total_billing_amount,cumulative_billing_events,cumulative_billing_amount,embryoscope_missing_payment
0,836188,2025-09,1,4.0,0,0.0,0.0,0.0,4.0
1,836188,2025-04,1,3.0,0,0.0,0.0,0.0,3.0
2,836188,2024-10,1,2.0,0,0.0,0.0,0.0,2.0
3,836188,2024-08,1,1.0,0,0.0,0.0,0.0,1.0


In [22]:
# Resumed embryoscope timeline data
print("=== RESUMED EMBRYOSCOPE TIMELINE DATA ===")
resumed_embryoscope_data = results_with_timeline['resumed_embryoscope_timeline']
print(f"Records: {len(resumed_embryoscope_data)}")
if not resumed_embryoscope_data.empty:
    display(resumed_embryoscope_data)
else:
    print("No resumed embryoscope timeline data found.")


=== RESUMED EMBRYOSCOPE TIMELINE DATA ===
Records: 1


,prontuario,period_month,embryoscope_usage_count,cumulative_embryoscope_usage,billing_events_count,total_billing_amount,cumulative_billing_events,cumulative_billing_amount,embryoscope_missing_payment
0,836188,2025-09,1,4.0,0,0.0,0.0,0.0,4.0


In [23]:
# Billing data
print("=== BILLING DATA EMBRYOSCOPE===")

condicao = mesclada_data['Descrição Gerencial'].isin([
    'Embryoscope'
])

print(f"Records: {len(mesclada_data[condicao])}")
mesclada_data[condicao]

=== BILLING DATA EMBRYOSCOPE===
Records: 0


,Cliente,Nome,Paciente,Nom Paciente,prontuario,DT Emissao,Descricao,Qntd.,Total,Descrição Gerencial,Loja,Tipo da nota,Numero,Serie Docto.,NF Eletr.,Vend. 1,Médico,Cliente_totvs,Operador,Produto,Valor Mercadoria,Custo,Custo Unit,Desconto,Unidade,Mês,Ano,Cta-Ctbl,Cta-Ctbl Eugin,Interno/Externo,Descrição Mapping Actividad,Ciclos,Qnt Cons.,Grp,Descr.TES,Lead Time,Data do Ciclo,Fez Ciclo?,line_number,extraction_timestamp,file_name


In [24]:
# Embryoscope treatment data
print("=== EMBRYOSCOPE TREATMENT DATA ===")
embryoscope_treatment_data = results_with_timeline['embryoscope_treatment']
print(f"Records: {len(embryoscope_treatment_data)}")
if not embryoscope_treatment_data.empty:
    display(embryoscope_treatment_data)
else:
    print("No embryoscope treatment data found.")


=== EMBRYOSCOPE TREATMENT DATA ===
Records: 31


,patient_PatientIDx,patient_PatientID,patient_FirstName,patient_LastName,patient_unit_huntington,embryo_KIDDate,embryo_FertilizationTime,treatment_TreatmentName,patient_unit_huntington_1
0,PC1P7BHG_45510.5102759375,836188,"MOURAO, CARLA T. S.",27/07/1990,Ibirapuera,NaT,2025-09-01 12:20:00,2025-1919,Ibirapuera
1,PC1P7BHG_45510.5102759375,836188,"MOURAO, CARLA T. S.",27/07/1990,Ibirapuera,NaT,2025-09-01 12:20:00,2025-1919,Ibirapuera
2,PC1P7BHG_45510.5102759375,836188,"MOURAO, CARLA T. S.",27/07/1990,Ibirapuera,NaT,2025-09-01 12:20:00,2025-1919,Ibirapuera
3,PC1P7BHG_45510.5102759375,836188,"MOURAO, CARLA T. S.",27/07/1990,Ibirapuera,NaT,2025-09-01 12:20:00,2025-1919,Ibirapuera
4,PC1P7BHG_45510.5102759375,836188,"MOURAO, CARLA T. S.",27/07/1990,Ibirapuera,NaT,2025-09-01 12:20:00,2025-1919,Ibirapuera
5,PC1P7BHG_45510.5102759375,836188,"MOURAO, CARLA T. S.",27/07/1990,Ibirapuera,NaT,2025-09-01 12:20:00,2025-1919,Ibirapuera
6,PC1P7BHG_45510.5102759375,836188,"MOURAO, CARLA T. S.",27/07/1990,Ibirapuera,2025-04-28,2025-04-21 13:50:00,2025 - 763,Ibirapuera
7,PC1P7BHG_45510.5102759375,836188,"MOURAO, CARLA T. S.",27/07/1990,Ibirapuera,2025-04-28,2025-04-21 13:50:00,2025 - 763,Ibirapuera
8,PC1P7BHG_45510.5102759375,836188,"MOURAO, CARLA T. S.",27/07/1990,Ibirapuera,2025-04-28,2025-04-21 13:50:00,2025 - 763,Ibirapuera
9,PC1P7BHG_45510.5102759375,836188,"MOURAO, CARLA T. S.",27/07/1990,Ibirapuera,2025-04-28,2025-04-21 13:50:00,2025 - 763,Ibirapuera


# 4a. Mensalidades

In [25]:
# # Comprehensive embryo timeline data
# print("=== COMPREHENSIVE EMBRYO TIMELINE DATA ===")
# comprehensive_embryo_timeline_data = results_with_timeline['comprehensive_embryo_timeline']
# print(f"Records: {len(comprehensive_embryo_timeline_data)}")
# if not comprehensive_embryo_timeline_data.empty:
#     display(comprehensive_embryo_timeline_data)
# else:
#     print("No comprehensive embryo timeline data found.")


# 4b. Taxa Cryo

In [26]:
# Cryopreservation events timeline data
print("=== CRYOPRESERVATION EVENTS TIMELINE DATA ===")
cryopreservation_events_data = results_with_timeline['cryopreservation_events_timeline']
print(f"Records: {len(cryopreservation_events_data)}")
if not cryopreservation_events_data.empty:
    display(cryopreservation_events_data)
else:
    print("No cryopreservation events timeline data found.")


=== CRYOPRESERVATION EVENTS TIMELINE DATA ===
Records: 3


,prontuario,period_month,freezing_events_count,cumulative_freezing_events,billing_events_count,total_billing_amount,cumulative_billing_events,cumulative_billing_amount,events_missing_payment
0,836188,2025-04,1.0,2.0,0.0,0.0,2.0,4940.0,0.0
1,836188,2024-10,1.0,1.0,1.0,2470.0,2.0,4940.0,-1.0
2,836188,2024-07,0.0,0.0,1.0,2470.0,1.0,2470.0,-1.0


In [27]:
# Cryopreservation events timeline data
print("=== RESUMED CRYOPRESERVATION EVENTS TIMELINE DATA ===")
resumed_cryopreservation_events_data = results_with_timeline['resumed_cryopreservation_events_timeline']
print(f"Records: {len(resumed_cryopreservation_events_data)}")
if not resumed_cryopreservation_events_data.empty:
    display(resumed_cryopreservation_events_data)
else:
    print("No resumed cryopreservation events timeline data found.")

=== RESUMED CRYOPRESERVATION EVENTS TIMELINE DATA ===
Records: 1


,prontuario,period_month,freezing_events_count,cumulative_freezing_events,billing_events_count,total_billing_amount,cumulative_billing_events,cumulative_billing_amount,events_missing_payment
0,836188,2025-04,1.0,2.0,0.0,0.0,2.0,4940.0,0.0


In [28]:
# Billing data
print("=== BILLING DATA TAXA CRYO===")

condicao = (mesclada_data['Descricao'].str.startswith('CRIOPRESERVACAO')) | (mesclada_data['Descrição Gerencial']=='Coleta - Crio')

print(f"Records: {len(mesclada_data[condicao])}")
mesclada_data[condicao]

=== BILLING DATA TAXA CRYO===
Records: 2


,Cliente,Nome,Paciente,Nom Paciente,prontuario,DT Emissao,Descricao,Qntd.,Total,Descrição Gerencial,Loja,Tipo da nota,Numero,Serie Docto.,NF Eletr.,Vend. 1,Médico,Cliente_totvs,Operador,Produto,Valor Mercadoria,Custo,Custo Unit,Desconto,Unidade,Mês,Ano,Cta-Ctbl,Cta-Ctbl Eugin,Interno/Externo,Descrição Mapping Actividad,Ciclos,Qnt Cons.,Grp,Descr.TES,Lead Time,Data do Ciclo,Fez Ciclo?,line_number,extraction_timestamp,file_name
34,836189,HERALDO FALCAO,836188,CARLA TAINA SOARES MOURAO,836188,2024-10-24,CRIOPRESERVACAO MEDICOS INTERNOS,1.0,2470.0,Criopreservação,10101,502,224181,RPS,222785,10,EDUARDO LEME ALVES MOTTA,878871,,S3566.04,0.0,0.0,0.0,0.0,Ibirapuera,0,0,31110100020,,,Criopreservação,0,0,1,PRESTACAO DE SERVICO,None,NaT,False,322271,20251023_201129,Mescladas2 (4).xlsx
49,836188,CARLA TAINA SO,836188,CARLA TAINA SOARES MOURAO,836188,2024-07-29,CRIOPRESERVACAO MEDICOS INTERNOS,1.0,2470.0,Criopreservação,10101,502,210893,RPS,209540,10,EDUARDO LEME ALVES MOTTA,878870,,S3566.04,0.0,0.0,0.0,0.0,Ibirapuera,0,0,31110100020,,,Criopreservação,0,0,1,PRESTACAO DE SERVICO,None,NaT,False,307454,20251023_201129,Mescladas2 (4).xlsx


In [29]:
# Embryo freezing data
print("=== EMBRYO FREEZING DATA ===")
congelamentos_embrioes_data = results_with_timeline['congelamentos_embrioes']
print(f"Records: {len(congelamentos_embrioes_data)}")
if not congelamentos_embrioes_data.empty:
    display(congelamentos_embrioes_data)
else:
    print("No embryo freezing data found.")


=== EMBRYO FREEZING DATA ===
Records: 2


,id,CodCongelamento,Unidade,prontuario,paciente,Data,Hora,Ciclo,CicloRecongelamento,condicoes_amostra,empresa_transporte,clinica_origem,responsavel_recebimento,responsavel_recebimento_data,responsavel_armazenamento,responsavel_armazenamento_data,NEmbrioes,NPailletes,Identificacao,Tambor,Cane,Cane2,Tecnica,Ovulo,D2,D3,D4,D5,D6,D7,rack,rack2,rack3,rack4,Observacoes,BiologoResponsavel,BiologoFIV,BiologoFIV2,status_financeiro,responsavel_congelamento_d5,responsavel_checagem_d5,responsavel_congelamento_d6,responsavel_checagem_d6,responsavel_congelamento_d7,responsavel_checagem_d7,hash,extraction_timestamp
0,28623,E954/25,1,836188,esposa,2025-04-28,1900-01-01 11:34:00,1250/25,Não,None,None,None,<NA>,NaT,<NA>,NaT,1,1,E22430,None,None,None,Vitrificação,NaN,NaN,NaN,NaN,NaN,NaN,1.0,None,NaN,NaN,NaN,None,<NA>,Beatriz Aiello,Izadora Reis,Cobrar,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,031ee5bf784c44606f30f93f00c740fc,2025-07-21 21:38:47
1,27090,E2847/24,1,836188,esposa,2024-10-31,1900-01-01 10:40:00,3610/24,Não,None,None,None,<NA>,NaT,<NA>,NaT,1,1,E21680,None,None,None,Vitrificação,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,None,<NA>,Bruna Lázaro Lourenço,Beatriz Aiello,Cobrar,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,3d730dafc7cbfa2d243c546d5f96805c,2025-07-21 21:38:47


In [30]:
# Oocyte freezing data
print("=== OOCYTE FREEZING DATA ===")
congelamentos_ovulos_data = results_with_timeline['congelamentos_ovulos']
print(f"Records: {len(congelamentos_ovulos_data)}")
if not congelamentos_ovulos_data.empty:
    display(congelamentos_ovulos_data)
else:
    print("No oocyte freezing data found.")


=== OOCYTE FREEZING DATA ===
Records: 0
No oocyte freezing data found.


In [31]:
# Semen freezing data
print("=== SEMEN FREEZING DATA ===")
congelamentos_semen_data = results_with_timeline['congelamentos_semen']
print(f"Records: {len(congelamentos_semen_data)}")
if not congelamentos_semen_data.empty:
    display(congelamentos_semen_data)
else:
    print("No semen freezing data found.")


=== SEMEN FREEZING DATA ===
Records: 0
No semen freezing data found.


# 5 Consultas

In [32]:
# Consultations timeline data
print("=== CONSULTATIONS TIMELINE DATA ===")
consultas_timeline_data = results_with_timeline['consultas_timeline']
print(f"Records: {len(consultas_timeline_data)}")
if not consultas_timeline_data.empty:
    display(consultas_timeline_data)
else:
    print("No consultations timeline data found.")


=== CONSULTATIONS TIMELINE DATA ===
Records: 1


,prontuario,period_month,consultas_events_count,cumulative_consultas_events,billing_events_count,total_billing_amount,total_quantity,cumulative_billing_events,cumulative_billing_amount,cumulative_quantity,consultas_missing_payment
0,836188.0,2024-05,2,2.0,1,873.0,1.0,1.0,873.0,1.0,1.0


In [33]:
# Resumed consultations timeline data
print("=== RESUMED CONSULTATIONS TIMELINE DATA ===")
resumed_consultas_data = results_with_timeline['resumed_consultas_timeline']
print(f"Records: {len(resumed_consultas_data)}")
if not resumed_consultas_data.empty:
    display(resumed_consultas_data)
else:
    print("No resumed consultations timeline data found.")


=== RESUMED CONSULTATIONS TIMELINE DATA ===
Records: 1


,prontuario,period_month,consultas_events_count,cumulative_consultas_events,billing_events_count,total_billing_amount,total_quantity,cumulative_billing_events,cumulative_billing_amount,cumulative_quantity,consultas_missing_payment
0,836188.0,2024-05,2,2.0,1,873.0,1.0,1.0,873.0,1.0,1.0


In [34]:
# Billing data
print("=== BILLING DATA cCONSULTAS===")

condicao = mesclada_data['Descrição Gerencial'].isin([
    'Consultas de CRH',
    'Outras Consultas'
])

print(f"Records: {len(mesclada_data[condicao])}")
mesclada_data[condicao]

=== BILLING DATA cCONSULTAS===
Records: 1


,Cliente,Nome,Paciente,Nom Paciente,prontuario,DT Emissao,Descricao,Qntd.,Total,Descrição Gerencial,Loja,Tipo da nota,Numero,Serie Docto.,NF Eletr.,Vend. 1,Médico,Cliente_totvs,Operador,Produto,Valor Mercadoria,Custo,Custo Unit,Desconto,Unidade,Mês,Ano,Cta-Ctbl,Cta-Ctbl Eugin,Interno/Externo,Descrição Mapping Actividad,Ciclos,Qnt Cons.,Grp,Descr.TES,Lead Time,Data do Ciclo,Fez Ciclo?,line_number,extraction_timestamp,file_name
52,836189,HERALDO FALCAO,836188,CARLA TAINA SOARES MOURAO,836188,2024-05-17,PRIMEIRA CONSULTA REPRODUCAO HUMANA,1.0,873.0,Consultas de CRH,10101,502,200249,RPS,198920,10,EDUARDO LEME ALVES MOTTA,878871,,S3555.01,0.0,0.0,0.0,97.0,Ibirapuera,0,0,31110100009,,,1ª Consulta,0,0,1,PRESTACAO DE SERVICO,73,2024-07-29,True,254285,20251023_201129,Mescladas2 (4).xlsx


In [35]:
# Example: Query multiple prontuarios
# prontuarios = ['160751', '171805', '876950']
# all_results = {}
# 
# for pront in prontuarios:
#     print(f"\nQuerying prontuario: {pront}")
#     all_results[pront] = query_patient_data(pront)
# 
# # Compare results
# for pront, results in all_results.items():
#     print(f"Prontuario {pront}: {len(results['tratamentos'])} tratamentos")
